In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Library 

In [3]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 4.5 MB/s 


In [5]:
pip install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 663.1 MB 1.9 kB/s 
     |████████████████████████████████| 8.8 MB 35.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.2.0+cu92 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.2.0+cu92 which 

In [6]:
# pip install pyvi

# Pre-process

In [8]:
# Original ViMMRC Preprocessing
#=========================================#
import glob
import os
import json
from nltk.tokenize import sent_tokenize, word_tokenize
# from pyvi import ViTokenizer

import nltk
nltk.download('punkt')

def sent_word_tokenize(text):
    s = s.lower()
    s = str(s)

    sents = sent_tokenize(s)
    words = [word_tokenize(s) for s in sents]
    return words

def pre_process_per_sent(sent):

    s = ' '.join([str(elem) for elem in sent]) 
    s = s.lower()
    s = str(s) 
    s = word_tokenize(s)  
    return s

def pre_process_per_doc(text):
    result = []
    for sent in text:
        s = pre_process_per_sent(sent)
        
        result.append(s)
    return result

def preprocess(task):
    print('Preprocessing the dataset ' + task + '...')
    dataset_names = ['train_race.json', 'dev_race.json', 'test_race.json']
    q_id = 0
    label_dict = {'A':0, 'B':1, 'C':2, 'D':3}
    for dataset_name in dataset_names:
        data_all = []
        path = os.path.join('drive/My Drive/CODE/ViMMRC/dataset/race_ver_2/', dataset_name)
        with open(path, 'r', encoding='utf-8') as fpr:
            data_raws = json.load(fpr)
            for data_raw in data_raws:
                artical = pre_process_per_doc([word_tokenize(s.strip()) for s in sent_tokenize(data_raw['article'])])
                    
                for i in range(len(data_raw['questions'])):
                    instance = {}
                    instance['article'] = artical
                    instance['question'] = pre_process_per_sent(data_raw['questions'][i].split(' '))
                    instance['ground_truth'] = label_dict[ data_raw['answers'][i] ]
                        
                    a = pre_process_per_sent(data_raw['options'][i][0].split(' '))
                    b = pre_process_per_sent(data_raw['options'][i][1].split(' '))
                    c = pre_process_per_sent(data_raw['options'][i][2].split(' '))
                    d = pre_process_per_sent(data_raw['options'][i][3].split(' '))
                        
                    instance['options'] = [a, b, c, d]
                        
                    
                    instance['q_id'] = q_id

                    try:
                        
                        instance['types'] = data_raw['types'][i]
                    except Exception as e:
                        print(data_raw)
                        print(data_raw['types'])
                        # raise e

                    
                    # instance['filename'] = filename
                        
                    q_id += 1
                    data_all.append(instance)
                    if len(data_all) % 1000==0:
                        print(len(data_all))

        with open(os.path.join('drive/My Drive/CODE/ViMMRC/dataset/comatch', task, 'sequence', dataset_name), 'w', encoding='utf8') as fpw:
            json.dump(data_all, fpw, ensure_ascii=False)

if __name__ == '__main__':
    preprocess('race')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Preprocessing the dataset race...
1000
2000
{'answers': ['A', 'D', 'B', 'D', 'D', 'D'], 'options': [['Thiếu lễ độ với mẹ', 'Nói dối mẹ', 'Không thương mẹ', 'Giận dỗi mẹ'], ['Tức giận', 'Buồn bực', 'Đau xót', 'Cả A và C'], ['Nói trực tiếp trước mặt En-ri-cô', 'Viết thư cho En-ri-cô', 'Nhờ cô giáo nhắn nhủ En-ri-cô', 'Ngồi tâm sự với En-ri-cô'], ['Vì bố gợi lại những kỉ niệm giữa mẹ và En-ri-cô', 'Vì En-ri-cô sợ bố', 'Vì En-ri-cô thấy xấu hổ trước lời nói chân tình của bố', 'Cả A và C'], ['Người bố muốn con phải đọc kĩ, suy ngẫm, tự rút ra bài học cho bản thân', 'Cách giữ thể diện cho người bị phê bình', 'Thể hiện bố En-ri-cô là người tinh tế, tâm lí, sâu sắc', 'Cả 3 đáp án trên'], ['Sẵn sàng bỏ một năm hạnh phúc để tránh cho con một giờ đau khổ', 'Thức suốt đêm, cúi mình trên chiếc nôi trông chừng hơi thở hổn hển của con', 'Người mẹ có thể ăn xin để nuôi con, thậm chí có thể hi sinh cả tính mạng', 'Là người mẹ nhân hậu, bao dung, hết lòng yêu thương con']], 'questions': ['Nhân vật En-ri

# Main 

## Train 

In [ ]:
# https://github.com/shuohangwang/comatch

import argparse
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import json
import os
import glob

path_dataset = 'drive/My Drive/CODE/ViMMRC/dataset/comatch'

# parser = argparse.ArgumentParser(description='Multiple Choice Reading Comprehension')
# parser.add_argument('--task', type=str, default='race',
#                     help='task name')
# parser.add_argument('--model', type=str, default='CoMatch',
#                     help='model name')
# parser.add_argument('--emb_dim', type=int, default=300,
#                     help='size of word embeddings')
# parser.add_argument('--mem_dim', type=int, default=150,
#                     help='hidden memory size')
# parser.add_argument('--lr', type=float, default=0.002,
#                     help='initial learning rate')
# parser.add_argument('--epochs', type=int, default=30,
#                     help='upper epoch limit')
# parser.add_argument('--batch_size', type=int, default=10, metavar='N',
#                     help='batch size')
# parser.add_argument('--dropoutP', type=float, default=0.2,
#                     help='dropout ratio')
# parser.add_argument('--seed', type=int, default=1111,
#                     help='random seed')
# parser.add_argument('--cuda', action='store_true',
#                     help='use CUDA')
# parser.add_argument('--interval', type=int, default=200, metavar='N',
#                     help='report interval')
# parser.add_argument('--exp_idx', type=str,  default='1',
#                     help='experiment index')
# parser.add_argument('--log', type=str,  default='nothing',
#                     help='take note')
# args = parser.parse_args()

args = {
    'task': 'race',
    'model': "CoMatch",
    'emb_dim': 1024,
    'mem_dim': 150,
    'lr': 0.002,
    'epochs': 30,
    'batch_size': 8,
    'dropoutP': 0.2,
    'seed': 1111,
    'cuda': True,
    'interval': 200,
    'exp_idx': '1',
    'log': 'nothing',
    'embedding': "drive/My Drive/CODE/ViMMRC/ELMO_ner.vec"
}


def accuracy(ground_truth, prediction):
    assert(len(ground_truth) == len(prediction))
    accuracy = float( (ground_truth==prediction).float().mean(0) )
    return accuracy

def evaluation(model, optimizer, criterion, corpus, cuda, batch_size, dataset='dev'):
    model.eval()
    labels_all = []
    pred_all = []
    total_loss = 0
    count = 0
    while True:
        data = corpus.get_batch(batch_size, dataset)
        output = model(data)
        _, pred = output.max(1)
        pred_all.append(pred.cpu())
        labels_all.append(data[3])
        labels = data[3].cuda() if cuda else data[3]
        loss = criterion(output, labels)

        loss = loss.detach()
        total_loss += float(loss * output.size(0))
        count += output.size(0)

        if corpus.start_id[dataset] >= len(corpus.data_all[dataset]): break


    loss = total_loss / count
    score = accuracy( torch.cat(labels_all), torch.cat(pred_all) )

    model.train()
    return score

def masked_softmax(vector, seq_lens):
    mask = vector.new(vector.size()).zero_()
    for i in range(seq_lens.size(0)):
        mask[i,:,:seq_lens[i]] = 1
    mask = Variable(mask, requires_grad=False)

    if mask is None:
        result = torch.nn.functional.softmax(vector, dim=-1)
    else:
        result = torch.nn.functional.softmax(vector * mask, dim=-1)
        result = result * mask
        result = result / (result.sum(dim=-1, keepdim=True) + 1e-13)
    return result

class MatchNet(nn.Module):
    def __init__(self, mem_dim, dropoutP):
        super(MatchNet, self).__init__()
        self.map_linear = nn.Linear(2*mem_dim, 2*mem_dim)
        self.trans_linear = nn.Linear(mem_dim, mem_dim)
        self.drop_module = nn.Dropout(dropoutP)

    def forward(self, inputs):
        proj_p, proj_q, seq_len = inputs
        trans_q = self.trans_linear(proj_q)
        att_weights = proj_p.bmm( torch.transpose(proj_q, 1, 2) )
        att_norm = masked_softmax(att_weights, seq_len)

        att_vec = att_norm.bmm(proj_q)
        elem_min = att_vec - proj_p
        elem_mul = att_vec * proj_p
        all_con = torch.cat([elem_min,elem_mul], 2)
        output = nn.ReLU()(self.map_linear(all_con))
        return output

class MaskLSTM(nn.Module):
    def __init__(self, in_dim, out_dim, layers=1, batch_first=True, bidirectional=True, dropoutP = 0.3):
        super(MaskLSTM, self).__init__()
        self.lstm_module = nn.LSTM(in_dim, out_dim, layers, batch_first=batch_first, bidirectional=bidirectional, dropout=dropoutP)
        self.drop_module = nn.Dropout(dropoutP)

    def forward(self, inputs):
        input, seq_lens = inputs
        mask_in = input.new(input.size()).zero_()
        for i in range(seq_lens.size(0)):
            mask_in[i,:seq_lens[i]] = 1
        mask_in = Variable(mask_in, requires_grad=False)

        input_drop = self.drop_module(input*mask_in)

        H, _ = self.lstm_module(input_drop)

        mask = H.new(H.size()).zero_()
        for i in range(seq_lens.size(0)):
            mask[i,:seq_lens[i]] = 1
        mask = Variable(mask, requires_grad=False)

        output = H * mask

        return output

class CoMatch(nn.Module):
    def __init__(self, corpus, args):
        super(CoMatch, self).__init__()
        self.emb_dim = args['emb_dim']
        self.mem_dim = args['mem_dim']
        self.dropoutP = args['dropoutP']
        self.cuda_bool = args['cuda']

        self.embs = nn.Embedding(len(corpus.dictionary), self.emb_dim)
        self.embs.weight.data.copy_(corpus.dictionary.embs)
        self.embs.weight.requires_grad = False

        self.encoder = MaskLSTM(self.emb_dim, self.mem_dim, dropoutP=self.dropoutP)
        self.l_encoder = MaskLSTM(self.mem_dim*8, self.mem_dim, dropoutP=self.dropoutP)
        self.h_encoder = MaskLSTM(self.mem_dim*2, self.mem_dim, dropoutP=0)

        self.match_module = MatchNet(self.mem_dim*2, self.dropoutP)
        self.rank_module = nn.Linear(self.mem_dim*2, 1)

        self.drop_module = nn.Dropout(self.dropoutP)

    def forward(self, inputs):
        documents, questions, options, _ = inputs
        d_word, d_h_len, d_l_len = documents
        o_word, o_h_len, o_l_len = options
        q_word, q_len = questions

        if self.cuda_bool: d_word, d_h_len, d_l_len, o_word, o_h_len, o_l_len, q_word, q_len = d_word.cuda(), d_h_len.cuda(), d_l_len.cuda(), o_word.cuda(), o_h_len.cuda(), o_l_len.cuda(), q_word.cuda(), q_len.cuda()
        d_embs = self.drop_module( Variable(self.embs(d_word), requires_grad=False) )
        o_embs = self.drop_module( Variable(self.embs(o_word), requires_grad=False) )
        q_embs = self.drop_module( Variable(self.embs(q_word), requires_grad=False) )

        d_hidden = self.encoder([d_embs.view(d_embs.size(0)*d_embs.size(1), d_embs.size(2), self.emb_dim), d_l_len.view(-1)] )
        o_hidden = self.encoder([o_embs.view(o_embs.size(0)*o_embs.size(1), o_embs.size(2), self.emb_dim), o_l_len.view(-1)])
        q_hidden = self.encoder([q_embs, q_len])

        d_hidden_3d = d_hidden.view(d_embs.size(0), d_embs.size(1) * d_embs.size(2), d_hidden.size(-1))
        d_hidden_3d_repeat = d_hidden_3d.repeat(1, o_embs.size(1), 1).view(d_hidden_3d.size(0)*o_embs.size(1), d_hidden_3d.size(1), d_hidden_3d.size(2))


        do_match = self.match_module([d_hidden_3d_repeat, o_hidden, o_l_len.view(-1)])
        dq_match = self.match_module([d_hidden_3d, q_hidden, q_len])

        dq_match_repeat = dq_match.repeat(1, o_embs.size(1), 1).view(dq_match.size(0)*o_embs.size(1), dq_match.size(1), dq_match.size(2))

        co_match= torch.cat([do_match, dq_match_repeat], -1)

        co_match_hier = co_match.view(d_embs.size(0)*o_embs.size(1)*d_embs.size(1), d_embs.size(2), -1)

        l_hidden = self.l_encoder([co_match_hier, d_l_len.repeat(1, o_embs.size(1)).view(-1)])
        l_hidden_pool, _ = l_hidden.max(1)

        h_hidden = self.h_encoder([l_hidden_pool.view(d_embs.size(0)*o_embs.size(1), d_embs.size(1), -1), d_h_len.view(-1, 1).repeat(1, o_embs.size(1)).view(-1)])
        h_hidden_pool, _ = h_hidden.max(1)

        o_rep = h_hidden_pool.view(d_embs.size(0), o_embs.size(1), -1)
        output = torch.nn.functional.log_softmax( self.rank_module(o_rep).squeeze(2) )

        return output

def prep_glove():
    vocab = {}
    ivocab = []
    tensors = []
    with open(args['embedding'], 'r', encoding='utf8') as f:
        for line in f:
            vals = line.rstrip().split(' ')
            if len(vals) != (args['emb_dim'] + 1):
                print(line)
                continue
            assert(len(vals) == (args['emb_dim'] + 1))
            word = vals[0]
            vec = torch.FloatTensor([ float(v) for v in vals[1:] ])
            vocab[word] = len(ivocab)
            ivocab.append(word)
            tensors.append(vec)
            assert (vec.size(0) == args['emb_dim'])
    assert len(tensors) == len(ivocab)
    tensors = torch.cat(tensors).view(len(ivocab), args['emb_dim'])
    with open('drive/My Drive/CODE/ViMMRC/dataset/race/embeddings.pt', 'wb') as fpw:
        torch.save([tensors, vocab, ivocab], fpw)


class Dictionary(object):
    def __init__(self, task):
        self.task = task
        filename = os.path.join(path_dataset, self.task, 'word2idx.pt')

        if os.path.exists(filename):
            self.word2idx = torch.load(os.path.join(path_dataset, self.task, 'word2idx.pt'))
            self.idx2word = torch.load(os.path.join(path_dataset, self.task, 'idx2word.pt'))
            self.word2idx_count = torch.load(os.path.join(path_dataset, self.task, 'word2idx_count.pt'))
        else:
            self.word2idx = {'<<padding>>':0, '<<unk>>':1}
            self.word2idx_count = {'<<padding>>':0, '<<unk>>':0}

            self.idx2word = ['<<padding>>', '<<unk>>']

            self.build_dict('train')
            self.build_dict('dev')
            if self.task != 'squad':
                self.build_dict('test')

            torch.save(self.word2idx, os.path.join(path_dataset, self.task, 'word2idx.pt'))
            torch.save(self.idx2word, os.path.join(path_dataset, self.task, 'idx2word.pt'))
            torch.save(self.word2idx_count, os.path.join(path_dataset, self.task, 'word2idx_count.pt'))

        filename_emb = os.path.join(path_dataset, task, 'embeddings.pt')
        print(filename_emb)
        if os.path.exists(filename_emb):
            self.embs = torch.load(filename_emb)
        else:
            self.embs = self.build_emb()

        print ("vocabulary size: " + str(len(self.idx2word)))

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = len(self.idx2word)
            self.idx2word.append(word)

            self.word2idx_count[word] = 1
        else:
            self.word2idx_count[word] += 1

        return self.word2idx[word]

    def build_dict(self, dataset):
        filename = os.path.join(path_dataset, self.task, 'sequence', dataset+ '_race' + '.json')

        if self.task == 'race':
            with open(filename, 'r', encoding='utf-8') as fpr:
                data_all = json.load(fpr)
                for instance in data_all:
                    words = instance['question']
                    for option in instance['options']: words += option
                    for sent in instance['article']: words += sent
                    for word in words: self.add_word(word)
        else:
            assert False, 'the task ' + self.task + ' is not supported!'

    def build_emb(self, all_vacob=False, filter=False, threshold=10):
        word2idx = torch.load(os.path.join(path_dataset, self.task, 'word2idx.pt'))
        idx2word = torch.load(os.path.join(path_dataset, self.task, 'idx2word.pt'))
        emb = torch.FloatTensor(len(idx2word), args['emb_dim']).zero_()
        print ("Loading Glove ...")
        print ("Raw vacabulary size: " + str(len(idx2word)) )

        if not os.path.exists('drive/My Drive/CODE/ViMMRC/dataset/race/embeddings.pt'): 
            prep_glove()
        glove_tensors, glove_vocab, glove_ivocab = torch.load('drive/My Drive/CODE/ViMMRC/dataset/race/embeddings.pt')

        if not all_vacob:
            self.word2idx = {'<<padding>>':0, '<<unk>>':1}
            self.idx2word = ['<<padding>>', '<<unk>>']
        count = 0
        for w_id, word in enumerate(idx2word):
            if word in glove_vocab:
                id = self.add_word(word)
                emb[id] = glove_tensors[glove_vocab[word]]
                count += 1
        emb = emb[:len(self.idx2word)]

        print("Number of words not appear in glove: " + str(len(idx2word)-count) )
        print ("Vocabulary size: " + str(len(self.idx2word)))
        torch.save(emb, os.path.join(path_dataset, self.task, 'embeddings.pt'))
        torch.save(self.word2idx, os.path.join(path_dataset, self.task, 'word2idx.pt'))
        torch.save(self.idx2word, os.path.join(path_dataset, self.task, 'idx2word.pt'))

        return emb

    def filter(self, threshold=10):
        for word, count in self.word2idx_count.items():
            if count > threshold and word not in self.word2idx:
                self.word2idx[word] = len(self.idx2word)
                self.idx2word.append(word)

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, task):
        self.task = task
        self.dictionary = Dictionary(task)

        self.data_all, self.start_id, self.indices = {}, {}, {}
        setnames = ['train', 'dev', 'test']
        for setname in setnames:
            self.data_all[setname] = self.load_data(os.path.join(path_dataset, self.task, 'sequence', setname) + '_race.json')
            print(setname, len(self.data_all[setname]))
            self.start_id[setname] = 0
            self.indices[setname] = torch.randperm(len(self.data_all[setname])) if setname == 'train' else torch.range(0, len(self.data_all[setname])-1)

    def seq2tensor(self, words):
        seq_tensor = torch.LongTensor(len(words))
        for id, word in enumerate(words):
            seq_tensor[id] = self.dictionary.word2idx[word] if word in self.dictionary.word2idx else 1
        return seq_tensor

    def load_data(self, filename):
        with open(filename, 'r', encoding='utf-8') as fpr:
            data = json.load(fpr)
        return data

    def get_batch(self, batch_size, setname):
        if self.start_id[setname] >= len(self.data_all[setname]):
            self.start_id[setname] = 0
            if setname == 'train': self.indices[setname] = torch.randperm(len(self.data_all[setname]))

        end_id = self.start_id[setname] + batch_size if self.start_id[setname] + batch_size < len(self.data_all[setname]) else len(self.data_all[setname])
        documents, questions, options, labels = [], [], [], []
        for i in range(self.start_id[setname], end_id):
            instance_id = int(self.indices[setname][i])

            instance = self.data_all[setname][instance_id]

            questions.append(instance['question'])
            options.append(instance['options'])
            documents.append(instance['article'])
            labels.append(instance['ground_truth'])

        self.start_id[setname] += batch_size

        questions = self.seq2tensor(questions)
        documents = self.seq2Htensor(documents)
        options = self.seq2Htensor(options)
        labels = torch.LongTensor(labels)
        return [documents, questions, options, labels]

    def seq2tensor(self, sents, sent_len_bound=50):
        sent_len_max = max([len(s) for s in sents])
        sent_len_max = min(sent_len_max, sent_len_bound)

        sent_tensor = torch.LongTensor(len(sents), sent_len_max).zero_()

        sent_len = torch.LongTensor(len(sents)).zero_()
        for s_id, sent in enumerate(sents):
            sent_len[s_id] = len(sent)
            for w_id, word in enumerate(sent):
                if w_id >= sent_len_max: break
                sent_tensor[s_id][w_id] = self.dictionary.word2idx.get(word, 1)
        return [sent_tensor, sent_len]

    def seq2Htensor(self, docs, sent_num_bound=50, sent_len_bound=50):
        sent_num_max = max([len(s) for s in docs])
        sent_num_max = min(sent_num_max, sent_num_bound)
        sent_len_max = max([len(w) for s in docs for w in s ])
        sent_len_max = min(sent_len_max, sent_len_bound)

        sent_tensor = torch.LongTensor(len(docs), sent_num_max, sent_len_max).zero_()
        sent_len = torch.LongTensor(len(docs), sent_num_max).zero_()
        doc_len = torch.LongTensor(len(docs)).zero_()
        for d_id, doc in enumerate(docs):
            doc_len[d_id] = len(doc)
            for s_id, sent in enumerate(doc):
                if s_id >= sent_num_max: break
                sent_len[d_id][s_id] = len(sent)
                for w_id, word in enumerate(sent):
                    if w_id >= sent_len_max: break
                    sent_tensor[d_id][s_id][w_id] = self.dictionary.word2idx.get(word, 1)
        return [sent_tensor, doc_len, sent_len]


if __name__ == '__main__':
    torch.manual_seed(args['seed'])
    if torch.cuda.is_available():
        if not args['cuda']:
            print("WARNING: You have a CUDA device, so you should probably run with --cuda")
        else:
            torch.cuda.manual_seed(args['seed'])

    corpus = Corpus(args['task'])
    model = eval(args['model'])(corpus, args)
    model.train()
    criterion = nn.NLLLoss()

    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = optim.Adamax(parameters, lr=args['lr'])


    if args['cuda']:
        model.cuda()
        criterion.cuda()

    start_time = time.time()
    total_loss = 0
    interval = args['interval']
    save_interval = len(corpus.data_all['train']) 

    best_dev_score = -99999
    # iterations = args['epochs']*len(corpus.data_all['train']) // args['batch_size']
    iterations = 3500

    print('max iterations: '+ str(iterations))
    for iter in range(iterations):
        optimizer.zero_grad()
        data = corpus.get_batch(args['batch_size'], 'train')
        output = model(data)
        labels = data[3].cuda() if args['cuda'] else data[3]
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += float(loss.data)

        if iter % interval == 0:
            cur_loss = total_loss / interval if iter!=0 else total_loss
            elapsed = time.time() - start_time
            print('| iterations {:3d} | start_id {:3d} | ms/batch {:5.2f} | loss {:5.3f}'.format(
            iter, corpus.start_id['train'], elapsed * 1000 / interval, cur_loss))
            total_loss = 0
            start_time = time.time()

        if iter % save_interval == 0:

            torch.save([model, optimizer, criterion], 'drive/My Drive/CODE/ViMMRC/model/comatch/'+args['task']+'_save.pt')
            score = evaluation(model, optimizer, criterion, corpus, args['cuda'], args['batch_size'])
            print('DEV accuracy: ' + str(score))

            with open('drive/My Drive/CODE/ViMMRC/model/comatch/'+args['task']+'_record.txt', 'a', encoding='utf-8') as fpw:
                if iter == 0: fpw.write(str(args) + '\n')
                fpw.write(str(iter) + ':\tDEV accuracy:\t' + str(score) + '\n')

            if score > best_dev_score:
                best_dev_score = score
                torch.save([model, optimizer, criterion], 'drive/My Drive/CODE/ViMMRC/model/comatch/'+args['task']+'_save_best.pt')

        if (iter+1) % (len(corpus.data_all['train']) // args['batch_size']) == 0:
            for param_group in optimizer.param_groups:
                param_group['lr'] *= 0.95


    model, optimizer, criterion = torch.load('drive/My Drive/CODE/ViMMRC/model/comatch/'+args['task']+'_save_best.pt')
    score = evaluation(model, optimizer, criterion, corpus, args['cuda'], args['batch_size'], dataset='test')
    with open('drive/My Drive/CODE/ViMMRC/model/comatch/'+args['task']+'_record.txt', 'a', encoding='utf-8') as fpw:
        fpw.write('TEST accuracy:\t' + str(score) + '\n')
    print('TEST accuracy: ' + str(score))

## Test 

In [ ]:
# TEST section

import argparse
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import json
import os
import glob

path_dataset = 'drive/My Drive/CODE/MMRC/dataset/comtach'

args = {
    'task': 'race',
    'model': "CoMatch",
    'emb_dim': 2392,
    'mem_dim': 150,
    'lr': 0.002,
    'epochs': 30,
    'batch_size': 8,
    'dropoutP': 0.2,
    'seed': 1111,
    'cuda': False,
    'interval': 200,
    'exp_idx': '1',
    'log': 'nothing',
    'embedding': "drive/My Drive/CODE/ViMMRC/ELMO_ner.vec",
    "result_file": "drive/My Drive/CODE/ViMMRC/dataset/comatch/result_ELMO.json",
    "model_dir": "drive/My Drive/CODE/ViMMRC/models/comatch/",
    "save_test": False,
    "dataset": "test"
}


def accuracy(ground_truth, prediction):
    assert(len(ground_truth) == len(prediction))
    accuracy = float( (ground_truth==prediction).float().mean(0) )
    return accuracy

# show result for each sentnce in test
def get_result(corpus, label, prediction, dataset):
    data = corpus.get_dataset(dataset)

    label = label.tolist()
    prediction = prediction.tolist()

    q_ids = []
    files_ids = []
    types = []
    for d in data:
        q_ids.append(d['q_id'])
        files_ids.append(d['filename'])
        types.append(d['types'])

    result = []
    for i in range(0, len(q_ids)):
        result.append({
            "qid": q_ids[i],
            "label": label[i],
            "prediction": prediction[i],
            "files_ids": files_ids[i],
            "types": types[i]
        })
    
    with open(args['result_file'], "w") as f:
        f.write(json.dumps(result))


def evaluation(model, optimizer, criterion, corpus, cuda, batch_size, dataset='dev'):
    model.eval()
    labels_all = []
    pred_all = []
    q_id = []
    total_loss = 0
    count = 0
    while True:
        data = corpus.get_batch(batch_size, dataset)
        output = model(data)
        _, pred = output.max(1)
        pred_all.append(pred.cpu())
        labels_all.append(data[3])
        labels = data[3].cuda() if cuda else data[3]
        loss = criterion(output, labels)

        loss = loss.detach()
        total_loss += float(loss * output.size(0))
        count += output.size(0)

        if corpus.start_id[dataset] >= len(corpus.data_all[dataset]): break

    loss = total_loss / count
    score = accuracy( torch.cat(labels_all), torch.cat(pred_all) )

    if args['save_test']:
        get_result(corpus, torch.cat(labels_all), torch.cat(pred_all), dataset)

    model.train()
    return score

def masked_softmax(vector, seq_lens):
    mask = vector.new(vector.size()).zero_()
    for i in range(seq_lens.size(0)):
        mask[i,:,:seq_lens[i]] = 1
    mask = Variable(mask, requires_grad=False)

    if mask is None:
        result = torch.nn.functional.softmax(vector, dim=-1)
    else:
        result = torch.nn.functional.softmax(vector * mask, dim=-1)
        result = result * mask
        result = result / (result.sum(dim=-1, keepdim=True) + 1e-13)
    return result

class MatchNet(nn.Module):
    def __init__(self, mem_dim, dropoutP):
        super(MatchNet, self).__init__()
        self.map_linear = nn.Linear(2*mem_dim, 2*mem_dim)
        self.trans_linear = nn.Linear(mem_dim, mem_dim)
        self.drop_module = nn.Dropout(dropoutP)

    def forward(self, inputs):
        proj_p, proj_q, seq_len = inputs
        trans_q = self.trans_linear(proj_q)
        att_weights = proj_p.bmm( torch.transpose(proj_q, 1, 2) )
        att_norm = masked_softmax(att_weights, seq_len)

        att_vec = att_norm.bmm(proj_q)
        elem_min = att_vec - proj_p
        elem_mul = att_vec * proj_p
        all_con = torch.cat([elem_min,elem_mul], 2)
        output = nn.ReLU()(self.map_linear(all_con))
        return output

class MaskLSTM(nn.Module):
    def __init__(self, in_dim, out_dim, layers=1, batch_first=True, bidirectional=True, dropoutP = 0.3):
        super(MaskLSTM, self).__init__()
        self.lstm_module = nn.LSTM(in_dim, out_dim, layers, batch_first=batch_first, bidirectional=bidirectional, dropout=dropoutP)
        self.drop_module = nn.Dropout(dropoutP)

    def forward(self, inputs):
        input, seq_lens = inputs
        mask_in = input.new(input.size()).zero_()
        for i in range(seq_lens.size(0)):
            mask_in[i,:seq_lens[i]] = 1
        mask_in = Variable(mask_in, requires_grad=False)

        input_drop = self.drop_module(input*mask_in)

        H, _ = self.lstm_module(input_drop)

        mask = H.new(H.size()).zero_()
        for i in range(seq_lens.size(0)):
            mask[i,:seq_lens[i]] = 1
        mask = Variable(mask, requires_grad=False)

        output = H * mask

        return output

class CoMatch(nn.Module):
    def __init__(self, corpus, args):
        super(CoMatch, self).__init__()
        self.emb_dim = args['emb_dim']
        self.mem_dim = args['mem_dim']
        self.dropoutP = args['dropoutP']
        self.cuda_bool = args['cuda']

        self.embs = nn.Embedding(len(corpus.dictionary), self.emb_dim)
        self.embs.weight.data.copy_(corpus.dictionary.embs)
        self.embs.weight.requires_grad = False

        self.encoder = MaskLSTM(self.emb_dim, self.mem_dim, dropoutP=self.dropoutP)
        self.l_encoder = MaskLSTM(self.mem_dim*8, self.mem_dim, dropoutP=self.dropoutP)
        self.h_encoder = MaskLSTM(self.mem_dim*2, self.mem_dim, dropoutP=0)

        self.match_module = MatchNet(self.mem_dim*2, self.dropoutP)
        self.rank_module = nn.Linear(self.mem_dim*2, 1)

        self.drop_module = nn.Dropout(self.dropoutP)

    def forward(self, inputs):
        documents, questions, options, _ = inputs
        d_word, d_h_len, d_l_len = documents
        o_word, o_h_len, o_l_len = options
        q_word, q_len = questions

        if self.cuda_bool: d_word, d_h_len, d_l_len, o_word, o_h_len, o_l_len, q_word, q_len = d_word.cuda(), d_h_len.cuda(), d_l_len.cuda(), o_word.cuda(), o_h_len.cuda(), o_l_len.cuda(), q_word.cuda(), q_len.cuda()
        d_embs = self.drop_module( Variable(self.embs(d_word), requires_grad=False) )
        o_embs = self.drop_module( Variable(self.embs(o_word), requires_grad=False) )
        q_embs = self.drop_module( Variable(self.embs(q_word), requires_grad=False) )

        d_hidden = self.encoder([d_embs.view(d_embs.size(0)*d_embs.size(1), d_embs.size(2), self.emb_dim), d_l_len.view(-1)] )
        o_hidden = self.encoder([o_embs.view(o_embs.size(0)*o_embs.size(1), o_embs.size(2), self.emb_dim), o_l_len.view(-1)])
        q_hidden = self.encoder([q_embs, q_len])

        d_hidden_3d = d_hidden.view(d_embs.size(0), d_embs.size(1) * d_embs.size(2), d_hidden.size(-1))
        d_hidden_3d_repeat = d_hidden_3d.repeat(1, o_embs.size(1), 1).view(d_hidden_3d.size(0)*o_embs.size(1), d_hidden_3d.size(1), d_hidden_3d.size(2))


        do_match = self.match_module([d_hidden_3d_repeat, o_hidden, o_l_len.view(-1)])
        dq_match = self.match_module([d_hidden_3d, q_hidden, q_len])

        dq_match_repeat = dq_match.repeat(1, o_embs.size(1), 1).view(dq_match.size(0)*o_embs.size(1), dq_match.size(1), dq_match.size(2))

        co_match= torch.cat([do_match, dq_match_repeat], -1)

        co_match_hier = co_match.view(d_embs.size(0)*o_embs.size(1)*d_embs.size(1), d_embs.size(2), -1)

        l_hidden = self.l_encoder([co_match_hier, d_l_len.repeat(1, o_embs.size(1)).view(-1)])
        l_hidden_pool, _ = l_hidden.max(1)

        h_hidden = self.h_encoder([l_hidden_pool.view(d_embs.size(0)*o_embs.size(1), d_embs.size(1), -1), d_h_len.view(-1, 1).repeat(1, o_embs.size(1)).view(-1)])
        h_hidden_pool, _ = h_hidden.max(1)

        o_rep = h_hidden_pool.view(d_embs.size(0), o_embs.size(1), -1)
        output = torch.nn.functional.log_softmax( self.rank_module(o_rep).squeeze(2) )

        return output

def prep_glove():
    vocab = {}
    ivocab = []
    tensors = []
    with open(args['embedding'], 'r', encoding='utf8') as f:
        for line in f:
            vals = line.rstrip().split(' ')
            if len(vals) != (args['emb_dim'] + 1):
                print(line)
                continue
            assert(len(vals) == (args['emb_dim'] + 1))
            word = vals[0]
            vec = torch.FloatTensor([ float(v) for v in vals[1:] ])
            vocab[word] = len(ivocab)
            ivocab.append(word)
            tensors.append(vec)
            assert (vec.size(0) == args['emb_dim'])
    assert len(tensors) == len(ivocab)
    tensors = torch.cat(tensors).view(len(ivocab), args['emb_dim'])
    with open('drive/My Drive/CODE/MMRC/dataset/race/embeddings.pt', 'wb') as fpw:
        torch.save([tensors, vocab, ivocab], fpw)


class Dictionary(object):
    def __init__(self, task):
        self.task = task
        filename = os.path.join(path_dataset, self.task, 'word2idx.pt')

        if os.path.exists(filename):
            self.word2idx = torch.load(os.path.join(path_dataset, self.task, 'word2idx.pt'))
            self.idx2word = torch.load(os.path.join(path_dataset, self.task, 'idx2word.pt'))
            self.word2idx_count = torch.load(os.path.join(path_dataset, self.task, 'word2idx_count.pt'))
        else:
            self.word2idx = {'<<padding>>':0, '<<unk>>':1}
            self.word2idx_count = {'<<padding>>':0, '<<unk>>':0}

            self.idx2word = ['<<padding>>', '<<unk>>']

            self.build_dict('train_race')
            self.build_dict('dev_race')
            if self.task != 'squad':
                self.build_dict('test_race')

            torch.save(self.word2idx, os.path.join(path_dataset, self.task, 'word2idx.pt'))
            torch.save(self.idx2word, os.path.join(path_dataset, self.task, 'idx2word.pt'))
            torch.save(self.word2idx_count, os.path.join(path_dataset, self.task, 'word2idx_count.pt'))

        filename_emb = os.path.join(path_dataset, task, 'embeddings.pt')
        print(filename_emb)
        if os.path.exists(filename_emb):
            self.embs = torch.load(filename_emb)
        else:
            self.embs = self.build_emb()

        print ("vocabulary size: " + str(len(self.idx2word)))

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = len(self.idx2word)
            self.idx2word.append(word)

            self.word2idx_count[word] = 1
        else:
            self.word2idx_count[word] += 1

        return self.word2idx[word]

    def build_dict(self, dataset):
        filename = os.path.join(path_dataset, self.task, 'sequence', dataset+'.json')

        if self.task == 'race':
            with open(filename, 'r', encoding='utf-8') as fpr:
                data_all = json.load(fpr)
                for instance in data_all:
                    words = instance['question']
                    for option in instance['options']: words += option
                    for sent in instance['article']: words += sent
                    for word in words: self.add_word(word)
        else:
            assert False, 'the task ' + self.task + ' is not supported!'

    def build_emb(self, all_vacob=False, filter=False, threshold=10):
        word2idx = torch.load(os.path.join(path_dataset, self.task, 'word2idx.pt'))
        idx2word = torch.load(os.path.join(path_dataset, self.task, 'idx2word.pt'))
        emb = torch.FloatTensor(len(idx2word), args['emb_dim']).zero_()
        print ("Loading Glove ...")
        print ("Raw vacabulary size: " + str(len(idx2word)) )

        if not os.path.exists('drive/My Drive/CODE/MMRC/dataset/race/embeddings.pt'): 
            prep_glove()
        glove_tensors, glove_vocab, glove_ivocab = torch.load('drive/My Drive/CODE/MMRC/dataset/race/embeddings.pt')

        if not all_vacob:
            self.word2idx = {'<<padding>>':0, '<<unk>>':1}
            self.idx2word = ['<<padding>>', '<<unk>>']
        count = 0
        for w_id, word in enumerate(idx2word):
            if word in glove_vocab:
                id = self.add_word(word)
                emb[id] = glove_tensors[glove_vocab[word]]
                count += 1
        emb = emb[:len(self.idx2word)]

        print("Number of words not appear in glove: " + str(len(idx2word)-count) )
        print ("Vocabulary size: " + str(len(self.idx2word)))
        torch.save(emb, os.path.join(path_dataset, self.task, 'embeddings.pt'))
        torch.save(self.word2idx, os.path.join(path_dataset, self.task, 'word2idx.pt'))
        torch.save(self.idx2word, os.path.join(path_dataset, self.task, 'idx2word.pt'))

        return emb

    def filter(self, threshold=10):
        for word, count in self.word2idx_count.items():
            if count > threshold and word not in self.word2idx:
                self.word2idx[word] = len(self.idx2word)
                self.idx2word.append(word)

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, task):
        self.task = task
        self.dictionary = Dictionary(task)

        self.data_all, self.start_id, self.indices = {}, {}, {}
        setnames = ['train', 'dev', 'test']
        for setname in setnames:
            self.data_all[setname] = self.load_data(os.path.join(path_dataset, self.task, 'sequence', setname) + '.json')
            print(setname, len(self.data_all[setname]))
            self.start_id[setname] = 0
            self.indices[setname] = torch.randperm(len(self.data_all[setname])) if setname == 'train' else torch.range(0, len(self.data_all[setname])-1)

    def get_dataset(self, dataset="dev"):
        return self.data_all[dataset]

    def seq2tensor(self, words):
        seq_tensor = torch.LongTensor(len(words))
        for id, word in enumerate(words):
            seq_tensor[id] = self.dictionary.word2idx[word] if word in self.dictionary.word2idx else 1
        return seq_tensor

    def load_data(self, filename):
        with open(filename, 'r', encoding='utf-8') as fpr:
            data = json.load(fpr)
        return data

    def get_batch(self, batch_size, setname):
        if self.start_id[setname] >= len(self.data_all[setname]):
            self.start_id[setname] = 0
            if setname == 'train': self.indices[setname] = torch.randperm(len(self.data_all[setname]))

        end_id = self.start_id[setname] + batch_size if self.start_id[setname] + batch_size < len(self.data_all[setname]) else len(self.data_all[setname])
        documents, questions, options, labels = [], [], [], []
        for i in range(self.start_id[setname], end_id):
            instance_id = int(self.indices[setname][i])

            instance = self.data_all[setname][instance_id]

            questions.append(instance['question'])
            options.append(instance['options'])
            documents.append(instance['article'])
            labels.append(instance['ground_truth'])

        self.start_id[setname] += batch_size

        questions = self.seq2tensor(questions)
        documents = self.seq2Htensor(documents)
        options = self.seq2Htensor(options)
        labels = torch.LongTensor(labels)
        return [documents, questions, options, labels]

    def seq2tensor(self, sents, sent_len_bound=50):
        sent_len_max = max([len(s) for s in sents])
        sent_len_max = min(sent_len_max, sent_len_bound)

        sent_tensor = torch.LongTensor(len(sents), sent_len_max).zero_()

        sent_len = torch.LongTensor(len(sents)).zero_()
        for s_id, sent in enumerate(sents):
            sent_len[s_id] = len(sent)
            for w_id, word in enumerate(sent):
                if w_id >= sent_len_max: break
                sent_tensor[s_id][w_id] = self.dictionary.word2idx.get(word, 1)
        return [sent_tensor, sent_len]

    def seq2Htensor(self, docs, sent_num_bound=50, sent_len_bound=50):
        sent_num_max = max([len(s) for s in docs])
        sent_num_max = min(sent_num_max, sent_num_bound)
        sent_len_max = max([len(w) for s in docs for w in s ])
        sent_len_max = min(sent_len_max, sent_len_bound)

        sent_tensor = torch.LongTensor(len(docs), sent_num_max, sent_len_max).zero_()
        sent_len = torch.LongTensor(len(docs), sent_num_max).zero_()
        doc_len = torch.LongTensor(len(docs)).zero_()
        for d_id, doc in enumerate(docs):
            doc_len[d_id] = len(doc)
            for s_id, sent in enumerate(doc):
                if s_id >= sent_num_max: break
                sent_len[d_id][s_id] = len(sent)
                for w_id, word in enumerate(sent):
                    if w_id >= sent_len_max: break
                    sent_tensor[d_id][s_id][w_id] = self.dictionary.word2idx.get(word, 1)
        return [sent_tensor, doc_len, sent_len]


if __name__ == '__main__':
    torch.manual_seed(args['seed'])
    if torch.cuda.is_available():
        if not args['cuda']:
            print("WARNING: You have a CUDA device, so you should probably run with --cuda")
        else:
            torch.cuda.manual_seed(args['seed'])

    corpus = Corpus(args['task'])
    model = eval(args['model'])(corpus, args)
    # model.train()
    criterion = nn.NLLLoss()

    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = optim.Adamax(parameters, lr=args['lr'])

    model, optimizer, criterion = torch.load(args['model_dir'] + args['task']+'_save_best.pt')
    score = evaluation(model, optimizer, criterion, corpus, args['cuda'], args['batch_size'], args['dataset'])
    # with open('drive/My Drive/CODE/MMRC/trainedmodel/'+args['task']+'_record.txt', 'a', encoding='utf-8') as fpw:
    #     fpw.write('TEST accuracy:\t' + str(score) + '\n')
    print('TEST accuracy: ' + str(score))

# Result analysis 

In [ ]:
# result analysis
import json

path_file = 'drive/My Drive/CODE/MMRC/dataset/result_MULTI_W_F_B_E.json'

with open(path_file, "r") as f:
    results = json.loads(f.read())

types = {
    'Matching': 0,
    'Paraphrasing': 0,
    'Single-sentence Reasoning': 0,
    'Multi-sentence Reasoning': 0,
    'Ambiguous Or Insufficient': 0
}

right = 0
wrong = 0

for r in results:
    if r['prediction'] != r['label']:
        types[r['types']] = types[r['types']] + 1
        wrong = wrong + 1
    else:
        right = right + 1

print(right)
print(wrong)
print(right/wrong)
for k,v in types.items():
    types[k] = v / len(results)

print(json.dumps(types, indent=4))

220
294
0.7482993197278912
{
    "Matching": 0.019455252918287938,
    "Paraphrasing": 0.02529182879377432,
    "Single-sentence Reasoning": 0.17120622568093385,
    "Multi-sentence Reasoning": 0.2782101167315175,
    "Ambiguous Or Insufficient": 0.07782101167315175
}


In [ ]:
MULTI: {
    "Matching": 0.019455252918287938,
    "Paraphrasing": 0.02529182879377432,
    "Single-sentence Reasoning": 0.17120622568093385,
    "Multi-sentence Reasoning": 0.2782101167315175,
    "Ambiguous Or Insufficient": 0.07782101167315175
}
ELMO: {
    "Matching": 0.023346303501945526,
    "Paraphrasing": 0.02529182879377432,
    "Single-sentence Reasoning": 0.15953307392996108,
    "Multi-sentence Reasoning": 0.26459143968871596,
    "Ambiguous Or Insufficient": 0.07003891050583658
}
Bert_Base: {
    "Matching": 0.027237354085603113,
    "Paraphrasing": 0.029182879377431907,
    "Single-sentence Reasoning": 0.17120622568093385,
    "Multi-sentence Reasoning": 0.2801556420233463,
    "Ambiguous Or Insufficient": 0.0914396887159533
}

FastText: {
    "Matching": 0.023346303501945526,
    "Paraphrasing": 0.02529182879377432,
    "Single-sentence Reasoning": 0.17120622568093385,
    "Multi-sentence Reasoning": 0.28793774319066145,
    "Ambiguous Or Insufficient": 0.08560311284046693
}

W2V_C2V: {
    "Matching": 0.029182879377431907,
    "Paraphrasing": 0.017509727626459144,
    "Single-sentence Reasoning": 0.17120622568093385,
    "Multi-sentence Reasoning": 0.31906614785992216,
    "Ambiguous Or Insufficient": 0.08949416342412451
}

W2V: {
    "Matching": 0.029182879377431907,
    "Paraphrasing": 0.017509727626459144,
    "Single-sentence Reasoning": 0.17120622568093385,
    "Multi-sentence Reasoning": 0.31906614785992216,
    "Ambiguous Or Insufficient": 0.08949416342412451
}